In [60]:
import re
import pandas as pd
from datetime import datetime

In [61]:
df = pd.read_csv("data/df_hack_final_processed.csv")
df = df.astype({'MEAS_DT': 'datetime64[ns]'})

In [62]:
interaction_graph = {
    "fm1" : [],
    "fm2" : ["Cu_1.1C", "Ni_1.1C", "Cu_1.2C", "Ni_1.2C"],
    "fm3" : ["Cu_2.1C", "Ni_2.1C", "Cu_2.2C", "Ni_2.2C"],
    "fm4" : ["Cu_2.1C", "Ni_2.1C", "Cu_2.2C", "Ni_2.2C", "Cu_3.1C", "Ni_3.1C", "Cu_3.2C", "Ni_3.2C", "Ni_6.1C", "Ni_6.2C"],
    "fm5" : ["Ni_4.1C", "Ni_4.1T"],
    "fm6" : ["Ni_5.1C", "Ni_5.1T", "Ni_5.2C", "Ni_5.2T"]
    }

In [63]:
def get_dataframe_per_fm(df, i, j, train=True):
    non_digit_features = [col for col in df.columns.tolist() if not re.findall(r'\d+', col)]
    filtered_columns_1 = [col for col in df.columns.tolist() if re.findall(f'{i}\\.{j}', col)]
    filtered_columns_2 = [col for col in df.columns.tolist() if re.findall(f'_{i}$', col)]
    add_features = []
    if train:
        add_features = interaction_graph[f'fm{i}']

    return df[non_digit_features+filtered_columns_1+filtered_columns_2 + add_features]

dfd = {f"fm{i}.{j}" : get_dataframe_per_fm(df, i, j) for j in range(1,3) for i in range(1,7)}

In [64]:
test_dataframe = pd.read_csv("data/test.csv")
test_dataframe = test_dataframe.astype({'MEAS_DT': 'datetime64[ns]'})

In [65]:
dfd_test = {f"fm{i}.{j}" : get_dataframe_per_fm(test_dataframe, i, j, False) for j in range(1, 3) for i in range(1, 7)}

In [66]:
def train_test_split(train_, test_):
    train = pd.concat([train_, test_]).drop_duplicates(subset='MEAS_DT', keep=False)
    test = pd.merge(test_, train_, how='inner', on='MEAS_DT', suffixes=('_y', '_X'))
    return {
        "train" : train, 
        "test" : test
        }

In [67]:
dfd_tt = {k : train_test_split(df_train_fm, dfd_test[k]) for k, df_train_fm in dfd.items()}

## CatBoost

In [68]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error
import numpy as np

In [69]:
from collections import defaultdict

In [70]:
models = {}
mae_values = defaultdict()
mape_values = defaultdict()

In [71]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [77]:
for floatmachine_name in dfd_tt.keys():

    train = dfd_tt[floatmachine_name]['train']
    test = dfd_tt[floatmachine_name]['test']

    output_features = [f.replace("_y", "") for f in test.columns.tolist() if "_y" in f]
    train = train.dropna(subset=output_features)

    for output_feature in output_features:
        print(f"training {floatmachine_name} : {output_feature}...")

        features_to_train_on = train.columns.tolist()
        features_to_train_on.remove('MEAS_DT')
        features_to_train_on = [f for f in features_to_train_on if not "min" in f and "max" not in f]

        train_pool = Pool(train[features_to_train_on], label=train[output_feature])
        test_pool = Pool(test[features_to_train_on], label=test[output_feature + "_X"])

        model = CatBoostRegressor(loss_function='MAE')

        model.fit(train_pool, eval_set=test_pool, use_best_model=True, plot=False, silent=True)

        train_predictions = model.predict(train_pool)
        test_predictions = model.predict(test_pool)

        train_mae = mean_absolute_error(train[output_feature], train_predictions)
        test_mae = mean_absolute_error(test[output_feature + "_X"], test_predictions)

        train_mape = mean_absolute_percentage_error(train[output_feature], train_predictions)
        test_mape = mean_absolute_percentage_error(test[output_feature + "_X"], test_predictions)

        mae_values[output_feature] = test_mae
        mape_values[output_feature] = test_mape

        print(f"Metrics for {floatmachine_name} : {output_feature}")
        print(f"Train MAE: {round(train_mae, 4)}, Train MAPE: {round(train_mape, 4)}%")
        print(f"Test MAE: {round(test_mae, 4)}, Test MAPE: {round(test_mape, 4)}%")
        print("----------------------------------//----------------------------------")

        models[f"{floatmachine_name}:{output_feature}"] = model

training fm1.1 : Ni_1.1C_min...
Metrics for fm1.1 : Ni_1.1C_min
Train MAE: 0.0764, Train MAPE: 2.9078%
Test MAE: 0.0716, Test MAPE: 2.6358%
----------------------------------//----------------------------------
training fm1.1 : Ni_1.1C_max...
Metrics for fm1.1 : Ni_1.1C_max
Train MAE: 0.0805, Train MAPE: 2.6062%
Test MAE: 0.0782, Test MAPE: 2.5392%
----------------------------------//----------------------------------
training fm1.1 : Cu_1.1C_min...
Metrics for fm1.1 : Cu_1.1C_min
Train MAE: 0.0417, Train MAPE: 0.9467%
Test MAE: 0.0416, Test MAPE: 0.9407%
----------------------------------//----------------------------------
training fm1.1 : Cu_1.1C_max...
Metrics for fm1.1 : Cu_1.1C_max
Train MAE: 0.0418, Train MAPE: 0.8851%
Test MAE: 0.0468, Test MAPE: 1.0041%
----------------------------------//----------------------------------
training fm2.1 : Cu_2.1T_min...
Metrics for fm2.1 : Cu_2.1T_min
Train MAE: 0.0041, Train MAPE: 1.0908%
Test MAE: 0.0044, Test MAPE: 1.3156%
----------------

In [78]:
mae_list = [v for _,v in mae_values.items()]
mape_list = [v for _,v in mape_values.items()]

print(f"Mean MAE {round(sum(mae_list) / len(mae_list), 4)}")
print(f"Mean MAPE {round(sum(mape_list) / len(mape_list), 4)}%")

Mean MAE 0.0561
Mean MAPE 2.2451%
